# COGS 118B - Final Project

# Finding Waldo 

## Group members

- Lillian Wood
- Taha Alam
- Zichen ‘Cardiff’ Jiang
- Will Lutz
- Sara Shao

# Introduction
In our project, we will perform PCA analysis on images of the character, Waldo, from Where’s Waldo to identify whether or not a test image is a picture of Waldo.

Utilizing a dataset of black and white images of Waldo, we will reduce the dimensionality of the vectorized images by calculating the eigenvalues and eigenvectors of the original image to reconstruct an image of Waldo in a smaller subspace. Once the eigenface is constructed using x number of principal components, we will be able to calculate the Euclidean distance between this constructed image and test images of both Waldo and non-Waldo projected into the same subspace to identify if the image is a picture of Waldo. The goal of this project is to elaborate on the topic of PCA that was discussed in class and present it in a fun and interactive way.

This project is important because finding Waldo manually is extremely tricky and time-consuming, sometimes ranging. Therefore, this intuitive PCA method to automatically find Waldo is needed.

# Related Work

Our project is based on the PCA example presented in class and in the homework. We will take the same steps to create the eigenfaces of our training and test sets of images. Once we create the eigenfaces for Waldo, we will follow the methods described in Liton Chandra Paul and Abdulla Al Sumam (2012) <a name="relatedwork"></a> [1](#relatedwork) in order to calculate the distance between the eigenfaces of the test images and the eigenfaces of the training images when projected into the subspace.

# Methods
Briefly, we collected 22 Waldo images, randomly vectorize 21 of them, and put these 21 vectors into a matrix. PCA will be done using these 21 images. The 22nd Waldo image will be used to test whether our eigenfaces can reconstruct this new image. Lastly, we tested our eigenfaces on a whole puzzle by breaking down the puzzle image to little square boxes and finding the box that has Waldo's face in it through reconstruction distance calculation.

### Collecting images
We collected images from various sources, including https://www.localguidesconnect.com/t5/General-Discussion/Alert-Answers-included-Waldo-Answers-if-you-guys-are-struggling/td-p/722677, https://petitefoxdesigns.wordpress.com/2015/11/05/wheres-waldo-wednesday-in-town/, https://github.com/wirooo/FolloWaldo, and so on. We whimsically selected some images from these websites, downloaded them, and manually croped them to a square where the tip of Waldo's right ear and Waldo's left cheekbone are roughly aligned at the same coordinates across cropped images. These images are named 1 to 22.jpg in `waldo_manual/`.

### Vectorizing Images
Each manually cropped images that roughly align with each other are of different pixel sizes. So we used `myimage.resize((x, x))` to resize each image to the same dimension so that the column vector representation of each image has the same size. 21 of the 22 Waldo images are used to form the input matrix that will be used to generate our eigenfaces.

### PCA
Explaination

### Reconstruction Distance Function
Explaination

# Results
Results - What did you discover? How well did it work?  As this is a class project, it is likely that many things did not work as well as planned.  For this project, detailing what went wrong is as important as describing what went well.  (approx 7 points)

### Preparing the images

vectorizing images explanation and code below

In [36]:
import numpy as np
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from sklearn.preprocessing import normalize
import time
from PIL import Image
import random
%matplotlib inline
import os

In [37]:
# Ex. waldo_matrix = vectorize(100, 21, 'waldo_manual')
# For the 22 Waldo pictures, 21 of them are turned into a 40000 by 1 (100 pixel x 100 pixel x 4) column vector. So each col of this 40000 by 21 matrix is one of the pictures.
def vectorize(pixel, pic_num, pic_dir):
    pic_dim = pixel * pixel * 4
    A = np.empty([pic_dim, pic_num])

    file_ls = []
    for filename in os.listdir(pic_dir):
        f = os.path.join(pic_dir, filename)
        file_ls.append(f)

    new_image_i = random.randrange(pic_num)
    for i in range(len(file_ls)):
        if i == new_image_i:
            continue
        else:
            f = file_ls[i]
            img = Image.open(f).convert('RGBA')
            img_resized = img.resize((100, 100))
            arr = np.array(img_resized)
            flat_arr = arr.ravel()
            v = np.matrix(flat_arr)
            col_v = v.T
            A[:, i-1:i] = col_v
            i += 1
    return A

In [38]:
# Ex. viewimage(waldo_input[:, 8], pixel)
def viewimage(vector, pixel):
    # Showing the 8th picture from the matrix
    vector_uint8 = vector.astype(np.uint8)
    shape = (pixel, pixel, 4)
    reconstruct_arr = np.asarray(vector_uint8).reshape(shape)
    reconstruct_img = Image.fromarray(reconstruct_arr, 'RGBA')
    reconstruct_img.show()

In [39]:
data = vectorize(100, 21, 'waldo_manual')
data.shape

(40000, 21)

In [40]:
viewimage(data[:, 18], 100)

### PCA

PCA explanation and code below

In [41]:
def eigsort(V, eigvals):
    
    # Sort the eigenvalues from largest to smallest. Store the sorted
    # eigenvalues in the column vector lambd.
    lohival = np.sort(eigvals)
    lohiindex = np.argsort(eigvals)
    lambd = np.flip(lohival)
    index = np.flip(lohiindex)
    Dsort = np.diag(lambd)
    
    # Sort eigenvectors to correspond to the ordered eigenvalues. Store sorted
    # eigenvectors as columns of the matrix vsort.
    M = np.size(lambd)
    Vsort = np.zeros((M, M))
    for i in range(M):
        Vsort[:,i] = V[:,index[i]]
    return Vsort, Dsort


In [42]:
# normc(M) normalizes the columns of M to a length of 1.
def normc(Mat):
    return normalize(Mat, norm='l2', axis=0)


In [43]:
meanface = np.mean(data, axis=1)
meanface = meanface[:, np.newaxis]
meanface

array([[149.66666667],
       [156.0952381 ],
       [125.19047619],
       ...,
       [134.66666667],
       [123.66666667],
       [255.        ]])

In [44]:
(np.matlib.repmat(meanface, 1, 21)).shape

(40000, 21)

In [45]:
data.shape

(40000, 21)

Subtract the mean from all of the data (using the command numpy.matlib.repmat), and call the matrix of mean-subtracted data A

In [46]:
A = data - np.matlib.repmat(meanface, 1, 21)

In [47]:
eigvals, Vold = np.linalg.eig(A.T.dot(A))

In [48]:
V, D = eigsort(Vold, eigvals)

In [49]:
U = A.dot(V)
#then normalize
U = normc(U)
U.shape

(40000, 21)

In [50]:
#PCA
C = U.T.dot(data - meanface)

In [51]:
# reconstruct the 1st Waldo image in the input matrix "data"
c=C[:,0]
c.shape

(21,)

In [53]:
## KERNEL DIED， dimension too big?
# Kernel didn't die on me. I got "ValueError: cannot reshape array of size 1600000000 into shape (100,100,4)"
# If transpose U first, U.T.dot(c), I get "ValueError: shapes (21,40000) and (21,) not aligned: 40000 (dim 1) != 21 (dim 0)"
xhat=U.T.dot(c) + meanface
viewimage(xhat, 100)

ValueError: shapes (21,40000) and (21,) not aligned: 40000 (dim 1) != 21 (dim 0)

### Reconstruction Distance Function

 explanation and code below

# Comparison to Related Work?

if we have time, write about existing github repo

# Discussion
What did you learn?  What could you do better? (What would you
have done next if you had more time)?.....  Why do you think it didn't work if it didn't?  
If everything worked perfectly,  what next steps would you suggest for follow-up work.  For full credit discuss two extensions or improvements to your project with short justifications for why you think that would work better (improvements) or why they are promising extensions. (approx 7 points) 

# Author Contribution
| Task      | Assignee |
| ----------- | ----------- |
| Introduction writing     | Will       |
| Related Work writing  | Will        |
| Finding puzzles with answers and cropping out Waldo | Cardiff and Lilly        |
| Vectorizing Images and methods writing   | Cardiff        |
| PCA and methods writing      | Sara       |
| Reconstruction Distance Function and methods writing   | Lilly and Taha        |
| Comparison to Related Work writing      |    []    |
| Discussion writing  |  all     |
| Editing | all |

# Footnotes

<a name="relatedwork"></a>1.[^](#relatedwork): last name, first name. (year). name of article. article link<br>